## Bulk Delete Registered Data Source

This notebook uses the Purview Python SDK to delete a registered data source and <b>all</b> of its associated assets. Support for the data sources ADLS Gen2 and Azure SQL DB are supported. The prefix of an assets fully qualified name is used to query the catalog and determine the list of assets being deleted. The prefix is derived from the registered data source endpoint field.

**Warning:**
This notebook has the potential to delete a lot of assets all at once. Verify what will be deleted first before running the delete sequence in the last cell.


In [ ]:
import os
import math
from azure.purview.scanning import PurviewScanningClient
from azure.purview.catalog import PurviewCatalogClient
from azure.purview.administration.account import PurviewAccountClient
from azure.identity import ClientSecretCredential 
from azure.core.exceptions import HttpResponseError

In [ ]:
client_id = "" 
client_secret = ""
tenant_id = ""
purview_account_name = ""
registered_data_source_name = ""

In [ ]:
credentials = ClientSecretCredential(client_id=client_id, client_secret=client_secret, tenant_id=tenant_id)
client_catalog = PurviewCatalogClient(endpoint=f"https://{purview_account_name}.purview.azure.com/", credential=credentials, logging_enable=True)
client_scan = PurviewScanningClient(endpoint=f"https://{purview_account_name}.scan.purview.azure.com", credential=credentials, logging_enable=True)

In [ ]:
switch_endpoint_kind = {
    "AzureSqlDatabase":"serverEndpoint",
    "AdlsGen2":"endpoint"
}

In [ ]:
try:
    response = client_scan.data_sources.delete(registered_data_source_name)
    endpoint_kind = switch_endpoint_kind.get(response['kind'])
    source_endpoint = response['properties'][endpoint_kind].rstrip("/")
    print(source_endpoint)
except HttpResponseError as e:
    print(e)

In [ ]:
page_limit = 1000

body_input={
    "limit": page_limit,
    "offset": 0,
    "filter": {
        "attributeName": "qualifiedName",
        "operator": "startswith",
        "attributeValue": source_endpoint
      }
}

In [ ]:
def delete_asset(x):
    if x['qualifiedName'].startswith(source_endpoint):
            client_catalog.entity.delete_by_guid(x['id'])

In [ ]:
result_count = client_catalog.discovery.query(search_request=body_input)['@search.count']
number_of_offsets = int(math.ceil(result_count / page_limit))

for x in range(number_of_offsets):
    body_input['offset'] = x
    response_value = client_catalog.discovery.query(search_request=body_input)['value']
    list(map(delete_asset,response_value))